In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from find_screen_utils import *

#Add Gaze Position to Video

In [2]:
gaze = pd.read_csv('images/tobii/screen_record.csv')
vid_sync = pd.read_csv('images/tobii/screen_record_video_sync.csv')

In [3]:
vid_sync.head()

,Unnamed: 0,VTS time,VTS validity
0,546224547,0,0
1,606211099,59985355,0


In [4]:
gaze['Vid_Time'] = (gaze['index'] - 546224547)/1000000

In [5]:
gaze = gaze[gaze.Vid_Time > 0] # only start tracking eyes once video starts

In [32]:
infile = 'images/tobii/fullstream.mp4'
outfile = 'images/tobii/fullstream_gaze.m4v'

In [36]:
vid = cv2.VideoCapture(infile)

size = (1920, 1080)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordiantes

i = 0
gaze_val = gaze['gaze position validity'].values
gaze_x = gaze['gaze position x'].values
gaze_y = gaze['gaze position y'].values

while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        for _ in range(2):
            frame_cp = frame.copy()
            if gaze_val[i] == 0:
                gp_x = gaze_x[i]
                gp_y = gaze_y[i]
                cv2.circle(frame_cp, (int(1920*gp_x), int(1080*gp_y)), 20, [0,255,0], 2)
            out.write(frame_cp)
            i += 1
    else:
        break


vid.release()
out.release()